In [1]:
%load_ext autoreload
%autoreload 2

In [41]:
import pandas as pd
import missingno as msn
import numpy as np
from src.loan_pred.preprocessing.preprocess import convert_dtype

In [42]:
train_prevloans = pd.read_csv("../data/raw_data/train/trainprevloans.csv")
train_prevloans.head()

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,closeddate,referredby,firstduedate,firstrepaiddate
0,8a2a81a74ce8c05d014cfb32a0da1049,301682320,2,2016-08-15 18:22:40.000000,2016-08-15 17:22:32.000000,10000.0,13000.0,30,2016-09-01 16:06:48.000000,NaN,2016-09-14 00:00:00.000000,2016-09-01 15:51:43.000000
1,8a2a81a74ce8c05d014cfb32a0da1049,301883808,9,2017-04-28 18:39:07.000000,2017-04-28 17:38:53.000000,10000.0,13000.0,30,2017-05-28 14:44:49.000000,NaN,2017-05-30 00:00:00.000000,2017-05-26 00:00:00.000000
2,8a2a81a74ce8c05d014cfb32a0da1049,301831714,8,2017-03-05 10:56:25.000000,2017-03-05 09:56:19.000000,20000.0,23800.0,30,2017-04-26 22:18:56.000000,NaN,2017-04-04 00:00:00.000000,2017-04-26 22:03:47.000000
3,8a8588f35438fe12015444567666018e,301861541,5,2017-04-09 18:25:55.000000,2017-04-09 17:25:42.000000,10000.0,11500.0,15,2017-04-24 01:35:52.000000,NaN,2017-04-24 00:00:00.000000,2017-04-24 00:48:43.000000
4,8a85890754145ace015429211b513e16,301941754,2,2017-06-17 09:29:57.000000,2017-06-17 08:29:50.000000,10000.0,11500.0,15,2017-07-14 21:18:43.000000,NaN,2017-07-03 00:00:00.000000,2017-07-14 21:08:35.000000


In [43]:
train_prevloans.shape

(18183, 12)

In [44]:
cols_dtypes = {
    "customerid": "category",
    "systemloanid": "int",
    "loannumber": "int",
    "approveddate": "datetime",
    "creationdate": "datetime",
    "loanamount": "float",
    "totaldue": "float",
    "termdays": "int",
    "closeddate": "datetime",
    "referredby": "category",
    "firstduedate": "datetime",
    "firstrepaiddate": "datetime"
}
train_prevloans = convert_dtype(data=train_prevloans, columns_type=cols_dtypes)
train_prevloans.dtypes

customerid               category
systemloanid                int32
loannumber                  int32
approveddate       datetime64[ns]
creationdate       datetime64[ns]
loanamount                float64
totaldue                  float64
termdays                    int32
closeddate         datetime64[ns]
referredby               category
firstduedate       datetime64[ns]
firstrepaiddate    datetime64[ns]
dtype: object

In [45]:
train_prevloans.isnull().sum()

customerid             0
systemloanid           0
loannumber             0
approveddate           0
creationdate           0
loanamount             0
totaldue               0
termdays               0
closeddate             0
referredby         17157
firstduedate           0
firstrepaiddate        0
dtype: int64

let's drop the referredby column

In [46]:
def preprocess_prev_loans(df):
    dt_cols = ["closeddate", "firstduedate", "firstrepaiddate"]
    col_to_select = ['customerid', 'loannumber', 'loanamount', 'totaldue', 'termdays']
    for col in dt_cols:
        df[f"{col}_days"] = (df[col] - df["approveddate"]).dt.days
        col_to_select.append(f"{col}_days")
    df = df[col_to_select]
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    return df

train_prevloans = preprocess_prev_loans(train_prevloans)
train_prevloans.head()

,customerid,loannumber,loanamount,totaldue,termdays,closeddate_days,firstduedate_days,firstrepaiddate_days
0,8a2a81a74ce8c05d014cfb32a0da1049,2,10000.0,13000.0,30,16,29,16
1,8a2a81a74ce8c05d014cfb32a0da1049,9,10000.0,13000.0,30,29,31,27
2,8a2a81a74ce8c05d014cfb32a0da1049,8,20000.0,23800.0,30,52,29,52
3,8a8588f35438fe12015444567666018e,5,10000.0,11500.0,15,14,14,14
4,8a85890754145ace015429211b513e16,2,10000.0,11500.0,15,27,15,27


In [49]:
from pathlib import Path
path = Path("../data/preprocessed/train/train_prevloans.csv")
path.parent.mkdir(parents=True, exist_ok=True)
train_prevloans.to_csv(path, index=False)